In [12]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import os
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pymongo

In [13]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\eweber\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [14]:
url = 'https://redplanetscience.com/'
browser.visit(url)

In [16]:
new_html = browser.html
soup = bs(html, 'html.parser')

In [18]:
news_title = soup.find('div', class_='content_title').text
print(news_title)

Nine Finalists Chosen in NASA's Mars 2020 Rover Naming Contest
